In [2]:
from tensorflow.keras import callbacks, optimizers
import sys
sys.path.append("../..")
from Shared.Data.yolov4_210.py_src.yolov4.tf import SaveWeightsCallback, YOLOv4
import time
import datetime

start = datetime.datetime.now()
print( start )

yolo = YOLOv4(tiny=True)
yolo.classes = "/nfs/Shared/Data/Wildtrack.names"
yolo.input_size = 608
yolo.batch_size = 32

yolo.make_model(activation1="relu")
yolo.load_weights(
    "/nfs/Shared/Data/yolov4-tiny.conv.29_AlexeyAB",
    weights_type="yolo"
)

train_data_set = yolo.load_dataset(
    "/nfs/Shared/Data/coco_train_only_person.txt",
    image_path_prefix="/nfs/Shared/Data/train2017",
    label_smoothing=0.05
)
val_data_set = yolo.load_dataset(
    "/nfs/Shared/Data/coco_val_only_person.txt",
    image_path_prefix="/nfs/Shared/Data/val2017",
    training=False
)

epochs = 400
lr = 1e-4

optimizer = optimizers.Adam(learning_rate=lr)
yolo.compile(optimizer=optimizer, loss_iou_type="ciou")

def lr_scheduler(epoch):
    if epoch < int(epochs * 0.5):
        return lr
    if epoch < int(epochs * 0.8):
        return lr * 0.5
    if epoch < int(epochs * 0.9):
        return lr * 0.1
    return lr * 0.01

_callbacks = [
    callbacks.LearningRateScheduler(lr_scheduler),
    callbacks.TerminateOnNaN(),
    callbacks.TensorBoard(
        log_dir="logs",
    ),
    SaveWeightsCallback(
        yolo=yolo, dir_path="trained",
        weights_type="yolo", epoch_per_save=1
    ),
]

yolo.fit(
    train_data_set,
    epochs=epochs,
    callbacks=_callbacks,
    validation_data=val_data_set,
    validation_steps=50,
    validation_freq=5,
    steps_per_epoch=100,
)

end = datetime.datetime.now()
print("start:" ,start)
print("end:" ,end)
print("total:" ,end - start)

2021-03-14 10:57:04.743477
Epoch 1/400
grid: 38*38 iou_loss: 18.2635727 conf_loss: 3111.00488 prob_loss: 3.62736988 total_loss 3132.896
grid: 19*19 iou_loss: 11.375 conf_loss: 712.015869 prob_loss: 2.52109838 total_loss 725.912
grid: 38*38 iou_loss: 15.4064522 conf_loss: 3071.63965 prob_loss: 3.07765961 total_loss 3090.12378
grid: 19*19 iou_loss: 14.8276043 conf_loss: 696.028687 prob_loss: 3.4249928 total_loss 714.28125
grid: 38*38 iou_loss: 13.5098896 conf_loss: 3036.79053 prob_loss: 2.60257769 total_loss 3052.90308
grid: 19*19 iou_loss: 9.06090736 conf_loss: 681.927917 prob_loss: 2.02975154 total_loss 693.018555
grid: 38*38 iou_loss: 16.8711605 conf_loss: 3002.45264 prob_loss: 3.45564055 total_loss 3022.7793
grid: 19*19 iou_loss: 13.699379 conf_loss: 666.80658 prob_loss: 3.04019809 total_loss 683.546204
grid: 38*38 iou_loss: 23.8037949 conf_loss: 2964.42749 prob_loss: 4.61020279 total_loss 2992.84131
grid: 19*19 iou_loss: 11.4235201 conf_loss: 655.604248 prob_loss: 2.5657289 total_lo

In [23]:
import sys
sys.path.append("../..")
from Shared.Data.yolov4_210.py_src.yolov4.tf import YOLOv4

yolo = YOLOv4(tiny=True, tpu=True)

yolo.classes = "/nfs/Shared/Data/Wildtrack.names"
yolo.input_size = (512, 384) # width, height
yolo.make_model(activation1="relu")

yolo.load_weights("trained/yolov4-tiny-155.weights", weights_type="yolo")

dataset = yolo.load_dataset(
    "/nfs/Shared/Data/coco_train_only_person.txt",
    training=False,
    image_path_prefix="/nfs/Shared/Data/train2017"
)

yolo.save_as_tflite(
    "convert_model/yolov4-tiny-155-relu-int8.tflite",
    quantization="full_int8",
    data_set=dataset,
    num_calibration_steps=400
)

INFO:tensorflow:Assets written to: /tmp/tmpm1fntgdr/assets


INFO:tensorflow:Assets written to: /tmp/tmpm1fntgdr/assets


In [24]:
!edgetpu_compiler -sa convert_model/yolov4-tiny-155-relu-int8.tflite -o convert_model

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 1415 ms.

Input model: convert_model/yolov4-tiny-155-relu-int8.tflite
Input size: 5.75MiB
Output model: convert_model/yolov4-tiny-155-relu-int8_edgetpu.tflite
Output size: 5.92MiB
On-chip memory used for caching model parameters: 5.69MiB
On-chip memory remaining for caching model parameters: 1.80MiB
Off-chip memory used for streaming uncached model parameters: 3.38KiB
Number of Edge TPU subgraphs: 3
Total number of operations: 132
Operation log: convert_model/yolov4-tiny-155-relu-int8_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 101
Number of operations that will run on CPU: 31

Operator                       Count 

In [1]:
import sys
sys.path.append("../..")
from yolov4_210.yolov4_210.py_src.yolov4.tflite import YOLOv4
import cv2

yolo = YOLOv4(tiny=True, tpu=True)

yolo.classes = "C:/Users/CSS-WU/Documents/GitHub/Machine_Learning/YoloV4/hhk7734/Wildtrack.names"

yolo.load_tflite("convert_model/yolov4-tiny-1-relu-int8_edgetpu.tflite")

yolo.inference(
    "C:/Users/CSS-WU/Desktop/cam6.mp4",
    is_image=False,
    cv_apiPreference=cv2.CAP_ANY,
    # cv_waitKey_delay= 25,
    cv_frame_size=(1920, 1080),
    cv_fourcc="YUYV",
)

ModuleNotFoundError: No module named 'yolov4_210'

In [1]:
import os
os.getcwd()

'C:\\Users\\CSS-WU\\Documents\\GitHub\\Machine_Learning\\YoloV4\\hhk7734\\NTUT_AI_Platform\\yolov4_210_coco_only_person_save_every_weight'